In [1]:
# Setup
%matplotlib inline
%load_ext autoreload
%autoreload 2
import warnings
import spacy
from optimized_anchor import anchor_text, anchor_base
import pickle
import myUtils
from myUtils import *
from dataset.dataset_loader import *
import datetime
%load_ext line_profiler

SEED = 84
torch.manual_seed(SEED)
warnings.simplefilter("ignore")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
# can be sentiment/spam/offensive
dataset_name = 'sentiment'
text_parser, label_parser, ds_train, ds_val = get_dataset(dataset_name)

Number of tokens in training samples: 6407
Number of tokens in training labels: 2


In [3]:
train, train_labels, test, test_labels, anchor_examples = preprocess_examples(ds_train)

In [4]:
nlp = spacy.load('en_core_web_sm')

In [6]:
ignored = get_ignored(anchor_examples)
normal_occurences = get_occurences(anchor_examples)
anchor_base.AnchorBaseBeam.best_group = BestGroup('check', normal_occurences, filter_anchors = False, desired_optimize = False)

## notice!

In [7]:
ignored = []

In [8]:
anchor_examples = anchor_examples[:20]

In [9]:
!nvidia-smi

Wed Aug 10 02:46:18 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.54       Driver Version: 510.54       CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Quadro RTX 6000     Off  | 00000000:AF:00.0 Off |                  Off |
| 33%   34C    P8    16W / 260W |      8MiB / 24576MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [10]:
optimize = True
anchor_text.AnchorText.set_optimize(optimize)
explainer = anchor_text.AnchorText(nlp, ['positive', 'negative'], use_unk_distribution=False)

In [11]:
from transformers import AutoTokenizer
model = torch.jit.load(f'models/tinybert/{dataset_name}/traced.pt').to(device)
model = model.eval()
tokenizer = AutoTokenizer.from_pretrained('huawei-noah/TinyBERT_General_4L_312D', use_fast = False)
myUtils.model = model
myUtils.tokenizer = tokenizer

In [13]:
#torch._C._jit_set_texpr_fuser_enabled(False)
my_utils = TextUtils(anchor_examples, test, explainer, predict_sentences, ignored, f"profile.pickle", optimize=optimize)
myUtils.model = model
myUtils.tokenizer = tokenizer
set_seed()
%prun -s cumtime -T check/profile.txt my_utils.compute_explanations(list(range(len(anchor_examples))))

number 0
[0.18181818181818182]
[0.2727272727272727]
[0.2727272727272727]
[0.7142857142857143]
[0.971830985915493]
[0.8571428571428571]
[0.5454545454545454]
[0.18181818181818182]
[0.36363636363636365]
number 1
[0.2727272727272727]
[0.2727272727272727]
[0.2727272727272727]
[0.18181818181818182]
[0.2727272727272727]
[0.45454545454545453]
[0.36363636363636365]
[0.18181818181818182]
[0.09090909090909091]
[0.45454545454545453]
[0.5454545454545454]
[0.2727272727272727]
[0.09090909090909091]
[0.36363636363636365]
number 2
[0.7142857142857143]
[0.6363636363636364]
[0.6666666666666666]
[0.36363636363636365]
[0.6363636363636364]
[0.6363636363636364]
[0.5454545454545454]
[0.36363636363636365]
[0.2727272727272727]
[0.6363636363636364]
[0.45454545454545453]
[0.5454545454545454]
[0.7619047619047619]
number 3
[0.36363636363636365]
[0.45454545454545453]
[0.3333333333333333]
[0.6666666666666666]
[0.0]
[0.8765432098765432]
[0.3076923076923077]
[0.45454545454545453]
[0.6363636363636364]
[0.090909090909090

         10627590 function calls (10625550 primitive calls) in 10.662 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000   10.700   10.700 {built-in method builtins.exec}
        1    0.000    0.000   10.700   10.700 <string>:1(<module>)
        1    0.001    0.001   10.700   10.700 myUtils.py:290(compute_explanations)
       20    0.023    0.001   10.685    0.534 myUtils.py:265(get_exp)
       20    0.002    0.000   10.662    0.533 anchor_text.py:220(explain_instance)
       20    0.005    0.000    9.474    0.474 anchor_base.py:255(anchor_beam)
      739    0.056    0.000    9.360    0.013 anchor_text.py:179(sample_fn)
      689    0.001    0.000    9.014    0.013 anchor_base.py:217(<lambda>)
      689    0.023    0.000    9.013    0.013 anchor_base.py:186(complete_sample_fn)
     5680    0.071    0.000    8.147    0.001 anchor_text.py:98(sample)
     5881    0.050    0.000    7.201    0.001 anch

In [13]:
###### my_utils = TextUtils(anchor_examples, test, explainer, predict_sentences, ignored,f"profile.pickle")
#%lprun -s -m modified_anchor.anchor_text -m modified_anchor.anchor_base -m myUtils -T profile.txt  my_utils.compute_explanations(list(range(len(anchor_examples))))

In [14]:
print(datetime.datetime.now())

2022-08-02 21:00:39.480614
